In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Only TCGA data

In [2]:
TCGA = pd.read_csv('TCGA_data.csv')
TCGA = TCGA.loc[(TCGA.cancer == 'BLCA') | (TCGA.cancer == 'PAAD')| (TCGA.cancer == 'KICH')| (TCGA.cancer == 'KIRC')| (TCGA.cancer == 'KIRP')]

In [4]:
TCGA.head()

,cyto.1p36.32,cyto.1p36.31,cyto.1p36.23,cyto.1p36.22,cyto.1p36.21,cyto.1p36.13,cyto.1p36.12,cyto.1p36.11,cyto.1p35.3,cyto.1p35.2,...,cyto.9q33.1,cyto.9q33.2,cyto.9q33.3,cyto.9q34.11,cyto.9q34.12,cyto.9q34.13,cyto.9q34.2,cyto.9q34.3,y,cancer
0,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,...,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,BC,BLCA
1,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,...,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,Normal,BLCA
2,0.4150,0.4150,0.4150,0.4150,0.4150,0.4150,0.4150,-0.2961,0.9639,-0.2736,...,-0.2716,-0.2716,-0.2716,-0.3456,-0.3456,0.2685,-0.3587,-0.3587,BC,BLCA
3,-0.0006,-0.0006,-0.0006,-0.0006,0.0033,0.0033,0.0033,0.0033,0.0033,0.0033,...,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,Normal,BLCA
4,-0.0130,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,...,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,Normal,BLCA


In [20]:
TCGA.shape # 786개

(2877, 788)

In [3]:
TCGA.loc[(TCGA.cancer == 'BLCA') & (TCGA.y != 'Normal'),'y'] = 'BC'
TCGA.loc[(TCGA.cancer == 'PAAD') & (TCGA.y != 'Normal'),'y'] = 'PC'
TCGA.loc[(TCGA.cancer == 'KICH') & (TCGA.y != 'Normal'),'y'] = 'RC'
TCGA.loc[(TCGA.cancer == 'KIRC') & (TCGA.y != 'Normal'),'y'] = 'RC'
TCGA.loc[(TCGA.cancer == 'KIRP') & (TCGA.y != 'Normal'),'y'] = 'RC'
TCGA.y.value_counts()

Normal    1403
RC         882
BC         408
PC         184
Name: y, dtype: int64

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

X = TCGA.drop(['cancer','y'], axis=1)
Y = TCGA['y']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

In [13]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [15]:
predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.912


## TCGA Model -> urine data

In [16]:
BC_100 = pd.read_csv('BC_32ea_k100_cyto.csv')
Normal_100 = pd.read_csv('Normal_21ea_k100_cyto.csv')
PC_100 = pd.read_csv('PC_20ea_k100_cyto.csv')
RC_100 = pd.read_csv('RC_20ea_k100_cyto.csv')

#drop person id
BC_100.drop(['Unnamed: 0','id'], axis = 1, inplace = True)
Normal_100.drop(['Unnamed: 0','id'], axis = 1, inplace = True)
PC_100.drop(['Unnamed: 0','id'], axis = 1, inplace = True)
RC_100.drop(['Unnamed: 0','id'], axis = 1, inplace = True)

BC_100['cancer'] = 'BC'
Normal_100['cancer'] = 'normal'
PC_100['cancer'] = 'PC'
RC_100['cancer'] = "RC"

K100 = pd.concat([BC_100,Normal_100,PC_100,RC_100], axis = 0)

del [BC_100,Normal_100,PC_100,RC_100]

K100 = K100.dropna(axis=0)
K100.isna().sum().sum()

0

In [37]:
a = list(TCGA.columns)
b = list(K100.columns)

new_col = []
for col in a :
    for col2 in b :
        if col==col2 :
            new_col.append(col)
len(new_col)            

763

In [38]:
new_col

['cyto.1p36.32',
 'cyto.1p36.31',
 'cyto.1p36.23',
 'cyto.1p36.22',
 'cyto.1p36.21',
 'cyto.1p36.13',
 'cyto.1p36.12',
 'cyto.1p36.11',
 'cyto.1p35.3',
 'cyto.1p35.2',
 'cyto.1p35.1',
 'cyto.1p34.3',
 'cyto.1p34.2',
 'cyto.1p34.1',
 'cyto.1p33',
 'cyto.1p32.3',
 'cyto.1p32.2',
 'cyto.1p32.1',
 'cyto.1p31.3',
 'cyto.1p31.2',
 'cyto.1p31.1',
 'cyto.1p22.3',
 'cyto.1p22.2',
 'cyto.1p22.1',
 'cyto.1p21.3',
 'cyto.1p21.2',
 'cyto.1p21.1',
 'cyto.1p13.3',
 'cyto.1p13.2',
 'cyto.1p13.1',
 'cyto.1p12',
 'cyto.1p11.2',
 'cyto.1q21.1',
 'cyto.1q21.2',
 'cyto.1q21.3',
 'cyto.1q22',
 'cyto.1q23.1',
 'cyto.1q23.2',
 'cyto.1q23.3',
 'cyto.1q24.1',
 'cyto.1q24.2',
 'cyto.1q24.3',
 'cyto.1q25.1',
 'cyto.1q25.2',
 'cyto.1q25.3',
 'cyto.1q31.1',
 'cyto.1q31.2',
 'cyto.1q31.3',
 'cyto.1q32.1',
 'cyto.1q32.2',
 'cyto.1q32.3',
 'cyto.1q41',
 'cyto.1q42.11',
 'cyto.1q42.12',
 'cyto.1q42.13',
 'cyto.1q42.2',
 'cyto.1q42.3',
 'cyto.1q43',
 'cyto.1q44',
 'cyto.10p15.3',
 'cyto.10p15.2',
 'cyto.10p15.1',
 'cyto

In [39]:
TCGA_new = TCGA[new_col]
K100_new = K100[new_col]

In [42]:
TCGA_new.shape

(2877, 763)

In [43]:
K100_new.shape

(88, 763)

In [44]:
TCGA_new.head()

,cyto.1p36.32,cyto.1p36.31,cyto.1p36.23,cyto.1p36.22,cyto.1p36.21,cyto.1p36.13,cyto.1p36.12,cyto.1p36.11,cyto.1p35.3,cyto.1p35.2,...,cyto.9q32,cyto.9q33.1,cyto.9q33.2,cyto.9q33.3,cyto.9q34.11,cyto.9q34.12,cyto.9q34.13,cyto.9q34.2,cyto.9q34.3,cancer
0,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,-0.1624,...,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,-0.1732,BLCA
1,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,...,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,0.0009,BLCA
2,0.4150,0.4150,0.4150,0.4150,0.4150,0.4150,0.4150,-0.2961,0.9639,-0.2736,...,-0.2716,-0.2716,-0.2716,-0.2716,-0.3456,-0.3456,0.2685,-0.3587,-0.3587,BLCA
3,-0.0006,-0.0006,-0.0006,-0.0006,0.0033,0.0033,0.0033,0.0033,0.0033,0.0033,...,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,0.0015,BLCA
4,-0.0130,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,0.0030,...,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,-0.0024,BLCA


In [48]:
K100_new.head()

,cyto.1p36.32,cyto.1p36.31,cyto.1p36.23,cyto.1p36.22,cyto.1p36.21,cyto.1p36.13,cyto.1p36.12,cyto.1p36.11,cyto.1p35.3,cyto.1p35.2,...,cyto.9q32,cyto.9q33.1,cyto.9q33.2,cyto.9q33.3,cyto.9q34.11,cyto.9q34.12,cyto.9q34.13,cyto.9q34.2,cyto.9q34.3,cancer
0,-0.025038,0.001059,-0.00455,0.013600,0.01136,-0.014229,-0.068324,-0.020417,-0.02185,-0.05465,...,-0.022250,0.015217,-0.044152,-0.004636,0.014333,0.0116,-0.004737,-0.019571,-0.022065,BC
1,0.057077,0.290471,0.18710,0.075943,0.05656,-0.038143,-0.066235,-0.022389,-0.01595,-0.02360,...,-0.060821,-0.043478,-0.046000,-0.007386,-0.009233,-0.0136,-0.060474,0.027714,0.002290,BC
2,0.069462,0.055706,0.06625,0.100914,0.05908,-0.023029,0.038471,0.087139,-0.07985,0.02390,...,-0.128250,-0.158783,-0.193576,-0.075977,-0.106667,-0.0114,-0.072421,-0.127714,-0.168903,BC
3,0.036308,0.025412,0.02550,0.052400,-0.00552,0.025286,-0.017000,-0.087639,0.04145,-0.02245,...,-0.077036,-0.019326,0.012515,-0.009591,0.016667,0.0438,0.019947,0.011500,0.031387,BC
4,0.181846,0.189353,0.29660,0.193000,0.24868,0.227943,0.253353,0.242028,0.20455,0.25615,...,-0.013250,-0.074109,0.024758,0.097000,0.018667,0.1170,0.058368,0.038571,0.020065,BC


In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

X = TCGA_new.drop(['cancer'], axis=1)
Y = TCGA['y']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3)

In [46]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [47]:
predicted = rf.predict(X_test)
accuracy = accuracy_score(y_test, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.889


In [49]:
X = K100_new.drop(['cancer'], axis=1)
Y = K100['cancer']

In [50]:
K100.shape # 765개

(88, 766)

In [51]:
predicted = rf.predict(X)
accuracy = accuracy_score(Y, predicted)

print(f'accuracy score: {accuracy:.3}')

accuracy score: 0.386
